In [1]:
!pip install h5py
!pip install Flask-CORS
from flask_cors import CORS



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import h5py
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# Load the K-Means model
model_file = 'Cluster_model.h5'  # Replace with the actual file path
with h5py.File(model_file, 'r') as f:
    # Load the cluster centers from the h5 file
    cluster_centers = np.array(f['cluster_centers'])

# Define your data as a pandas DataFrame (replace this with your actual data)
def predict_cluster(response_time, response_mark, attendance, user_response):
    data = pd.DataFrame({
        'Response Time': [response_time],  # Replace with your data
        'Response Marks': [response_mark],   # Replace with your data
        'Attendance': [attendance],      # Replace with your data
        'Response': [user_response]         # Replace with your data
    })

# Perform clustering based on the loaded cluster centers
    kmeans = KMeans(n_clusters=cluster_centers.shape[0], init=cluster_centers)

    cluster_labels = np.argmin(np.linalg.norm(data.values[:, np.newaxis] - cluster_centers, axis=2), axis=1)
    return cluster_labels

# Example: Print the cluster labels for each data point
for i, label in enumerate(predict_cluster(27,5,1,1)):
    print(f"Data point {i} is in cluster {label}")

Data point 0 is in cluster 0


In [3]:
from flask import Flask, request, jsonify
import json
import socket
app = Flask(__name__)

@app.route('/')
def index():
    return jsonify("Welcome to the home page")

@app.route('/predict_cluster', methods=['GET'])
def predict_cluster_route():
    #data = request.get_json()

    response_mark = request.args.get('response_mark')
    response_time = request.args.get('response_time')
    attendance = request.args.get('attendance')
    user_response = request.args.get('user_response')

    
    response_mark = int(response_mark)
    response_time = int(response_time)
    attendance = int(attendance)
    user_response = int(user_response)
     
    cluster = predict_cluster(response_time, response_mark, attendance, user_response)

    # Convert cluster to varchar based on conditions
    if cluster == 0:
        cluster_string = "Modarate"
    elif cluster == 1:
        cluster_string = "Good"
    elif cluster == 2:
        cluster_string = "Worst"
    else:
        cluster_string = "Unknown"  # You can provide a default value for other cases

#return jsonify(cluster_string)
    response = jsonify(cluster_string)
    response.headers.add("Access-Control-Allow-Origin", "http://localhost:3000")  # Adjust the origin as needed
    response.headers.add("Access-Control-Allow-Methods", "GET")
    return response

if __name__ == '__main__':
    # socket.getaddrinfo('http://127.0.0.1', 5001)
    app.run(host='localhost', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit


127.0.0.1 - - [21/Dec/2023 13:54:37] "GET /predict_cluster?response_mark=3&response_time=26&attendance=1&user_response=1 HTTP/1.1" 200 -
